In [2]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\TraficoAereoGt.csv")
rddfiltro = text_file.rdd.map(tuple)

In [4]:
#se toman los aeropuertos y aterrizajes 
rddAERO= rddfiltro.map(lambda word: (word[2],word[3]))
rddAERO.take(10)

[('Coatepeque', 3607),
 ('Coatepeque', 5261),
 ('Coatepeque', 629),
 ('Coatepeque', 3690),
 ('Coatepeque', 307),
 ('Coatepeque', 4064),
 ('Coatepeque', 2144),
 ('Coatepeque', 4469),
 ('Coatepeque', 3248),
 ('Coatepeque', 5078)]

In [5]:
#se suman todos los aterrizajes reduciendo por la clave aeropuerto
rddCONT=rddAERO.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONT.collect())

Conteo total -> [('Coatepeque', 167223), ('Huehuetenango', 198734), ('Poptun', 173559), ('Puerto Barros', 203204), ('Mundo Maya', 170385), ('La Aurora', 150513), ('Puerto San Jose', 162211), ('Quetzaltenango', 194403), ('Retalhuleu', 174374), ('San Marcos', 208806)]


In [11]:
#se orden los valores
rddCONT = rddCONT.sortBy(lambda a: a[1],False)

In [12]:
#se separan los nombre de los aeropuertos y totales
rddNombres = rddCONT.map(lambda x: (x[0]))
rddTotales = rddCONT.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['San Marcos', 'Puerto Barros', 'Huehuetenango', 'Quetzaltenango', 'Retalhuleu', 'Poptun', 'Mundo Maya', 'Coatepeque', 'Puerto San Jose', 'La Aurora']
[208806, 203204, 198734, 194403, 174374, 173559, 170385, 167223, 162211, 150513]


In [13]:
#se realiza la gráfica de barras
fig = go.Figure(data=go.Bar(x=rddNombres.collect(),y=rddTotales.collect()))
fig.update_layout(title_text='Total de aterrizajes por aeropuertos',title_font_size=30,
                  yaxis=dict(title='Total de Aterrizajes',title_font_size=25),
                  xaxis=dict(title='Nombre Cajero',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('R1_A2.html', auto_open=True)